# Neural Networks course project: #
# Google AudioSet sound classification with Deep Learning #
### Sapienza University of Rome ###

### by Ivan Senilov (1787618) ###

## 1. Introduction ##

This work represents a practical part of the Neural Networks course taught at Sapienza University.

The goal of this coursework is to:

1. Explore the [Google AudioSet](https://research.google.com/audioset/index.html)
2. Build the classification model based on Neural Network(s)
3. Validate the model
4. Evaluate and discuss the results

## 2. Audio classification problem ##

### 2.1 Audio features ###

Feature extraction is signal processing task of computing the numerical representation from the signal that can be used to characterize the audio segment \cite{prasad2007speech}.

Most of the audio features fall into three categories \cite{1221332}:

1. **Energy-based**. For example, 4Hz modulation energy used for speech/music classification \cite{scheirer1997construction}.
2. **Spectrum-based**. Examples of the category are roll-off of the spectrum, spectral flux, Mel Frequency Cepstral Coefficents (MFCC) \cite{scheirer1997construction} and linear spectrum pair, band periodicity \cite{lu2001content}.
3. **Perceptual based**, like pitch (estimated to discriminate songs and speech over music \cite{zhang1998content}).

The most developed areas of machine learning for audio classification include speech \cite{shrawankar2013techniques,lee2003optimizing,ghahremani2014pitch} and music \cite{logan2000mel,wang2006shazam,eronen2001comparison} recognition where  MFCCs are widely used as features. MFCCs were introduced in 1980 \cite{davis1980comparison} and showed better performance in terms of recognition of spoken words. However, when it comes to other types of sound recognition, the selection of feature extraction method becomes less obvious, even though MFCCs are used in, for example, application to environmental sound classification \cite{chu2009environmental,cotton2011spectral}.

<img src="figs/feat_extr.png">

Extraction pipelines for MPEG-7 (left) and MFCC (right) features (reprinted from \cite{1221332})

Typical approach of feature extraction process is to split the audio signal into small chunks of several $ms$ (exact size is domain-dependent) and feed it into computational function of one of many frameworks for extraction of features from audio (see Figure \ref{fig:feature_extraction} for examples of extraction pipelines for MPEG-7 \cite{casey2001mpeg} and MFCC features). The most popular frameworks include YAAFE \cite{mathieu2010yaafe} and openSMILE \cite{eyben2010opensmile,eyben2013recent}, which allows to extract following feature types:
\begin{itemize}
	\item Amplitude Modulation  \cite{eronen2001automatic}. Analyzed frequency ranges are: Tremolo (4 - 8 Hz) and Grain (10 - 40 Hz). For each of these ranges, it computes:
	\begin{itemize}
		\item Frequency of maximum energy in range
		\item Difference of the energy of this frequency and the mean energy over all frequencies
		\item Difference of the energy of this frequency and the mean energy in range
		\item Product of the two first values.
	\end{itemize}
	\item Autocorrelation coefficients $\mathit{ac}$ on each frame.
    $$ac(k)=\sum_{i=0}^{N-k-1}x(i)x(i+k),$$    
    where (here and below) $k$ is frame length in samples, $N$ is length of whole signal in samples and $x(i)$ is signal function.    
    \item Onset detection using a complex domain spectral flux method. \cite{duxbury2003complex}.    
    \item Energy $\mathit{en}$ as root mean square of an audio frame.
    $$en=\sqrt{\dfrac{\sum_{i=0}^{N-1}x(i)^2}{N}}$$    
    \item Envelope of an oscillating signal (smooth curve outlining its extremes).    
    \item Shape statistics (centroid, spread, skewness and kurtosis) of each frame’s Temporal Shape, Amplitude Envelope and Magnitude Spectrum.    
    \item Linear Predictor Coefficients (LPC) of a signal frame \cite{makhoul1975linear}.    
    \item Line Spectral Frequency (LSF) coefficients of a signal frame \cite{backstrom2006properties,schussler1976stability}.    
    \item Loudness coefficients \cite{moore1997model}.    
    \item Mel-frequencies cepstrum coefficients and Mel-frequencies spectrum.    
    \item Magnitude spectrum.    
    \item Octave band signal intensity (OBSI) using a triangular octave filter bank and  OBSI ratio between consecutive octave.    
    \item Sharpness and Spread of Loudness coefficients \cite{peeters2004large}.    
    \item Spectral crest factor per log-spaced band of 1/4 octave.    
    \item Spectral decrease, spectral flatness.    
    \item Spectral Flux.    
    \item Spectral roll-off (frequency so that 99\% of the energy is contained below) \cite{scheirer1997construction}.
    \item Spectral Slope (computed by linear regression of the spectral amplitude) \cite{peeters2004large}.    
    \item Spectral Variation (normalized correlation of spectrum between consecutive frames) \cite{peeters2004large}.    
    \item Zero-crossing rate (ZCR) for frame \cite{scheirer1997construction}.
\end{itemize}

## 3. Implementation ##

In this section, we will:

1. Load ontology and correspondence between video's ids and class labels.
2. Download the videos from Youtube to respective directories
3. Load labeled dataset into Numpy arrays
4. Build classifier based on Neural Network architecture
5. Evaluate it using cross-validation

First of all, import data manipulation libs.

In [2]:
import pandas as pd
import numpy as np

In [3]:
classes_of_interest = ("Vehicle", "Channel, environment and background", "Natural sounds")

with open("ontology.json", "r") as f:
    contents = f.read()

ontology = pd.read_json(contents)

ontology = ontology[["id", "name", "child_ids"]].set_index("id")

In [8]:
def extract_ids(cls, data):  # recursively add all child classes of input class to the list
    out = [] 
    for id, row in data.iterrows():
        if row["name"] == cls:
            out.append(id)
            if len(row["child_ids"]) > 0:
                for child in row["child_ids"]:
                    out.append(extract_ids(data["name"][child], data))
                
    return flattern(out)


def flattern(A):  # list flattening helper function
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flattern(i))
        else: rt.append(i)
    return rt

In [10]:
classes = {i:[] for i in classes_of_interest}
for cls in classes_of_interest:
    classes[cls] = extract_ids(cls, ontology)

In [11]:
train_data = pd.read_csv("balanced_train_segments.csv", skiprows=2, sep=", ", engine="python", index_col="# YTID")
train_data["positive_labels"] = train_data["positive_labels"].apply(lambda x: x.replace('\"','').split(","))

test_data = pd.read_csv("eval_segments.csv", skiprows=2, sep=", ", engine="python", index_col="# YTID")
test_data["positive_labels"] = test_data["positive_labels"].apply(lambda x: x.replace('\"','').split(","))

In [12]:
train_data.head()

,start_seconds,end_seconds,positive_labels
# YTID,,,
--PJHxphWEs,30.0,40.0,"[/m/09x0r, /t/dd00088]"
--ZhevVpy1s,50.0,60.0,[/m/012xff]
--aE2O5G5WE,0.0,10.0,"[/m/03fwl, /m/04rlf, /m/09x0r]"
--aO5cdqSAg,30.0,40.0,"[/t/dd00003, /t/dd00005]"
--aaILOrkII,200.0,210.0,"[/m/032s66, /m/073cg4]"


In [15]:
import youtube_dl
import os

def crop(start, length, filename):
    command = "ffmpeg -y -i " + filename + \
    " -ss  " + str(start) + " -t " + str(length) + \
    " -ac 1 -acodec copy " + filename.split(".")[0] + "_.wav"
    os.system(command)

to_download = {i:[] for i in classes_of_interest}
    
for cls in classes_of_interest:
    for id in classes[cls]:
        for row in train_data.itertuples(): 
            if row[3][0] == id:
                to_download[cls].append(row[0])
                

options = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav'
    }],
    'extractaudio' : True,
    'ignoreerrors' : True,
    'audioformat' : "wav",    # name the file the ID of the video
    'noplaylist' : True,    # only download single clip, not playlist
}
                
for cls in to_download:
    options['outtmpl'] = os.path.join(cls.replace(",","").replace(" ",""), '%(id)s.%(ext)s')
    for file in to_download[cls]:
        with youtube_dl.YoutubeDL(options) as ydl:
            ydl.download([file])
            filename = os.path.join(cls.replace(",","").replace(" ",""), file + ".wav")
            crop(train_data.loc[file]["start_seconds"],
                 10, filename)
            try:
                os.remove(filename)
            except OSError:
                pass


[youtube] yzCAEjPTnRI: Downloading webpage
[youtube] yzCAEjPTnRI: Downloading video info webpage
[youtube] yzCAEjPTnRI: Extracting video information
[youtube] yzCAEjPTnRI: Downloading MPD manifest
[youtube] yzCAEjPTnRI: Downloading MPD manifest
[dashsegments] Total fragments: 23
[download] Destination: Channelenvironmentandbackground/yzCAEjPTnRI.m4a
[download] 100% of 3.19MiB in 00:05.75MiB/s ETA 00:000003
[ffmpeg] Correcting container in "Channelenvironmentandbackground/yzCAEjPTnRI.m4a"
[ffmpeg] Destination: Channelenvironmentandbackground/yzCAEjPTnRI.wav
Deleting original file Channelenvironmentandbackground/yzCAEjPTnRI.m4a (pass -k to keep)
[youtube] QlqOy37nMqY: Downloading webpage
[youtube] QlqOy37nMqY: Downloading video info webpage
[youtube] QlqOy37nMqY: Extracting video information
[youtube] QlqOy37nMqY: Downloading MPD manifest
[download] Destination: Channelenvironmentandbackground/QlqOy37nMqY.m4a
[download] 100% of 18.78MiB in 00:0073MiB/s ETA 00:003
[ffmpeg] Correcting cont

[download] 100% of 4.55MiB in 00:0045MiB/s ETA 00:005
[ffmpeg] Correcting container in "Channelenvironmentandbackground/D-PxXM2I5gY.m4a"
[ffmpeg] Destination: Channelenvironmentandbackground/D-PxXM2I5gY.wav
Deleting original file Channelenvironmentandbackground/D-PxXM2I5gY.m4a (pass -k to keep)
[youtube] D5ebkJwpPrg: Downloading webpage
[youtube] D5ebkJwpPrg: Downloading video info webpage
[youtube] D5ebkJwpPrg: Extracting video information
[youtube] D5ebkJwpPrg: Downloading MPD manifest
[download] Destination: Channelenvironmentandbackground/D5ebkJwpPrg.webm
[download] 100% of 2.50MiB in 00:0056MiB/s ETA 00:002
[ffmpeg] Destination: Channelenvironmentandbackground/D5ebkJwpPrg.wav
Deleting original file Channelenvironmentandbackground/D5ebkJwpPrg.webm (pass -k to keep)
[youtube] DTXsmh43fzw: Downloading webpage
[youtube] DTXsmh43fzw: Downloading video info webpage
[youtube] DTXsmh43fzw: Extracting video information
[youtube] DTXsmh43fzw: Downloading MPD manifest
[download] Destination:

ERROR: I7DpbFk_Y94: YouTube said: This video contains content from brandon672008. It is not available in your country.


[youtube] Jc73va5hsQU: Downloading webpage
[youtube] Jc73va5hsQU: Downloading video info webpage
[youtube] Jc73va5hsQU: Extracting video information
[youtube] Jc73va5hsQU: Downloading MPD manifest
[download] Destination: Channelenvironmentandbackground/Jc73va5hsQU.webm
[download] 100% of 14.73MiB in 00:0070MiB/s ETA 00:007
[ffmpeg] Destination: Channelenvironmentandbackground/Jc73va5hsQU.wav
Deleting original file Channelenvironmentandbackground/Jc73va5hsQU.webm (pass -k to keep)
[youtube] KrmG43H1u70: Downloading webpage
[youtube] KrmG43H1u70: Downloading video info webpage
[youtube] KrmG43H1u70: Extracting video information
[youtube] KrmG43H1u70: Downloading MPD manifest
[download] Destination: Channelenvironmentandbackground/KrmG43H1u70.m4a
[download] 100% of 13.00MiB in 00:0060MiB/s ETA 00:005
[ffmpeg] Correcting container in "Channelenvironmentandbackground/KrmG43H1u70.m4a"
[ffmpeg] Destination: Channelenvironmentandbackground/KrmG43H1u70.wav
Deleting original file Channelenvironm

[youtube] u1qOtaWGDRo: Downloading MPD manifest
[download] Destination: Channelenvironmentandbackground/u1qOtaWGDRo.m4a
[download] 100% of 1.88MiB in 00:0075MiB/s ETA 00:002
[ffmpeg] Correcting container in "Channelenvironmentandbackground/u1qOtaWGDRo.m4a"
[ffmpeg] Destination: Channelenvironmentandbackground/u1qOtaWGDRo.wav
Deleting original file Channelenvironmentandbackground/u1qOtaWGDRo.m4a (pass -k to keep)
[youtube] vOiDwXo-SuM: Downloading webpage
[youtube] vOiDwXo-SuM: Downloading video info webpage
[youtube] vOiDwXo-SuM: Extracting video information
[youtube] vOiDwXo-SuM: Downloading MPD manifest
[download] Destination: Channelenvironmentandbackground/vOiDwXo-SuM.webm
[download] 100% of 5.61MiB in 00:0027MiB/s ETA 00:006
[ffmpeg] Destination: Channelenvironmentandbackground/vOiDwXo-SuM.wav
Deleting original file Channelenvironmentandbackground/vOiDwXo-SuM.webm (pass -k to keep)
[youtube] w4ayJNZ5w5o: Downloading webpage
[youtube] w4ayJNZ5w5o: Downloading video info webpage
[yo

...

ERROR: 6AZys4l4gbk: YouTube said: This video is not available.


[youtube] 73SYrra4rzM: Downloading webpage
[youtube] 73SYrra4rzM: Downloading video info webpage
[youtube] 73SYrra4rzM: Extracting video information
[youtube] 73SYrra4rzM: Downloading MPD manifest
[download] Destination: Naturalsounds/73SYrra4rzM.webm
[download] 100% of 4.97MiB in 00:0051MiB/s ETA 00:005
[ffmpeg] Destination: Naturalsounds/73SYrra4rzM.wav
Deleting original file Naturalsounds/73SYrra4rzM.webm (pass -k to keep)
[youtube] 8Y3pAqq5s0E: Downloading webpage
[youtube] 8Y3pAqq5s0E: Downloading video info webpage
[youtube] 8Y3pAqq5s0E: Extracting video information
[youtube] 8Y3pAqq5s0E: Downloading MPD manifest
[download] Destination: Naturalsounds/8Y3pAqq5s0E.webm
[download] 100% of 7.82MiB in 00:0076MiB/s ETA 00:009
[ffmpeg] Destination: Naturalsounds/8Y3pAqq5s0E.wav
Deleting original file Naturalsounds/8Y3pAqq5s0E.webm (pass -k to keep)
[youtube] 9ANnmOAB8js: Downloading webpage
[youtube] 9ANnmOAB8js: Downloading video info webpage
[youtube] 9ANnmOAB8js: Extracting video inf

[ffmpeg] Destination: Naturalsounds/XhUySw_CMg4.wav
Deleting original file Naturalsounds/XhUySw_CMg4.m4a (pass -k to keep)
[youtube] YfNxo2vkdUU: Downloading webpage
[youtube] YfNxo2vkdUU: Downloading video info webpage
[youtube] YfNxo2vkdUU: Extracting video information
[youtube] YfNxo2vkdUU: Downloading MPD manifest
[download] Destination: Naturalsounds/YfNxo2vkdUU.m4a
[download] 100% of 1.66MiB in 00:0091MiB/s ETA 00:001
[ffmpeg] Correcting container in "Naturalsounds/YfNxo2vkdUU.m4a"
[ffmpeg] Destination: Naturalsounds/YfNxo2vkdUU.wav
Deleting original file Naturalsounds/YfNxo2vkdUU.m4a (pass -k to keep)
[youtube] _XQ1rUrdIWg: Downloading webpage
[youtube] _XQ1rUrdIWg: Downloading video info webpage
[youtube] _XQ1rUrdIWg: Extracting video information
[youtube] _XQ1rUrdIWg: Downloading MPD manifest
[download] Destination: Naturalsounds/_XQ1rUrdIWg.m4a
[download] 100% of 4.30MiB in 00:0053MiB/s ETA 00:005
[ffmpeg] Correcting container in "Naturalsounds/_XQ1rUrdIWg.m4a"
[ffmpeg] Desti

ERROR: jWjqXjeGtHE: YouTube said: Please sign in to view this video.


[youtube] m5IvF38dyD8: Downloading webpage
[youtube] m5IvF38dyD8: Downloading video info webpage
[youtube] m5IvF38dyD8: Extracting video information
[youtube] m5IvF38dyD8: Downloading MPD manifest
[download] Destination: Naturalsounds/m5IvF38dyD8.webm
[download] 100% of 2.25MiB in 00:0081MiB/s ETA 00:002
[ffmpeg] Destination: Naturalsounds/m5IvF38dyD8.wav
Deleting original file Naturalsounds/m5IvF38dyD8.webm (pass -k to keep)
[youtube] mD-Ec0NsjIg: Downloading webpage
[youtube] mD-Ec0NsjIg: Downloading video info webpage
[youtube] mD-Ec0NsjIg: Extracting video information
[youtube] mD-Ec0NsjIg: Downloading MPD manifest
[download] Destination: Naturalsounds/mD-Ec0NsjIg.m4a
[download] 100% of 3.69MiB in 00:0005MiB/s ETA 00:004
[ffmpeg] Correcting container in "Naturalsounds/mD-Ec0NsjIg.m4a"
[ffmpeg] Destination: Naturalsounds/mD-Ec0NsjIg.wav
Deleting original file Naturalsounds/mD-Ec0NsjIg.m4a (pass -k to keep)
[youtube] miqwRA1QLw8: Downloading webpage
[youtube] miqwRA1QLw8: Downloading

In [4]:
import os
from scipy.io import wavfile
from librosa import feature

classes_dirs = [i.replace(",","").replace(" ","") for i in classes_of_interest[1:]]

feat = []
labels = []
class_num = 0
window = 4096
for directory in classes_dirs:
    count = 0
    for file in os.listdir(directory):
        filename = os.path.join(directory, file)
        if os.path.isfile(filename) and count < 100:
            count += 1
            rate, frames = wavfile.read(filename)
            if len(frames.shape) > 1:    # if stereo, take only one channel
                frames = frames[:,0]
            for i in range(0, len(frames)-window, int(window/2)):
                pxx = feature.mfcc(y=frames[i:i + window - 1], sr=rate, n_mfcc=20)
                # pxx = pxx[:,:850]
                # print(frames.shape, filename, pxx.shape, class_num)
                # if pxx.shape[1] == 850:
                feat.append(pxx)
                labels.append(class_num)
    class_num += 1
data = np.stack(feat)

from keras.utils import to_categorical
labels = to_categorical(labels)

Using TensorFlow backend.
/home/isenilov/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, accuracy_score
from keras.models import Sequential
from keras.layers import LSTM, Conv2D, Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.wrappers import Bidirectional
# from keras.callbacks import TensorBoard, ModelCheckpoint, Callback
from keras import optimizers, regularizers

from datetime import datetime



X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

print("\nTraining model...")
# architecture of the network is adopted from https://arxiv.org/pdf/1511.07035.pdf
model = Sequential()
# model.add(Bidirectional(LSTM(216, return_sequences=True, activation="relu",),
#                         input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Bidirectional(LSTM(216, return_sequences=True, activation="relu")))
# model.add(Bidirectional(LSTM(216, activation="relu")))
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding="same", activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(2))
model.add(Conv2D(filters=32, kernel_size=3))
model.add(GlobalAveragePooling2D())
model.add(Dense(2, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

hist = model.fit(X_train, y_train,
                 # callbacks=[tbCallback, mcCallback, testCallback0, testCallback1, testCallback2, testCallback3],
                 # validation_data=(X_1, y_1),
                 epochs=100,
                 batch_size=128,
                 verbose=1)


print("\nEvaluating...")
y_pred = to_categorical(model.predict_classes(X_test, verbose=1))
print(y_pred, y_test)
acc = accuracy_score(y_test, y_pred)
print("\nAccuracy:", acc)
rec = recall_score(y_test, y_pred, average="macro")
print("Recall:", rec)


Training model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 20, 8, 32)         320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 4, 32)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 2, 32)          9248      
_________________________________________________________________
global_average_pooling2d_2 ( (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 9,634
Trainable params: 9,634
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
35141/35141 [==============================] - 3s 85us/step - loss: 0.9629 - acc: 0.5921
Epoch 2/100
35141/35

35141/35141 [==============================] - 3s 84us/step - loss: 0.5299 - acc: 0.7335
Epoch 72/100
35141/35141 [==============================] - 3s 84us/step - loss: 0.5256 - acc: 0.7367
Epoch 73/100
35141/35141 [==============================] - 3s 84us/step - loss: 0.5342 - acc: 0.7305
Epoch 74/100
35141/35141 [==============================] - 3s 84us/step - loss: 0.5339 - acc: 0.7299
Epoch 75/100
35141/35141 [==============================] - 3s 84us/step - loss: 0.5273 - acc: 0.7342
Epoch 76/100
35141/35141 [==============================] - 3s 84us/step - loss: 0.5294 - acc: 0.7322
Epoch 77/100
35141/35141 [==============================] - 3s 84us/step - loss: 0.5322 - acc: 0.7276
Epoch 78/100
35141/35141 [==============================] - 3s 84us/step - loss: 0.5273 - acc: 0.7342
Epoch 79/100
35141/35141 [==============================] - 3s 84us/step - loss: 0.5309 - acc: 0.7287
Epoch 80/100
35141/35141 [==============================] - 3s 84us/step - loss: 0.5241 - acc: 

## x. References ##

1. Gemmeke, Jort F., et al. "Audio Set: An ontology and human-labeled dataset for audio events." *IEEE ICASSP*. 2017.	